In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from google.colab import drive

In [7]:
#input_data_sample = pd.read_excel("sample.xlsx")
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/DataLucipher/n2nlstm_code")

input_data_sample = pd.read_csv("combined_csv_all.csv")
valid_data_sample = pd.read_csv("combined_validation.csv")
test_data_sample = pd.read_csv("combined_test.csv")
input_data_sample.head(5)
valid_data_sample.head(5)
test_data_sample.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Text,Label
0,A woman called best buy to find out if she cou...,employment or career inquiries
1,A woman has been trying to find a power hornet...,product availability and stock
2,A customer has been speaking on the phone to a...,change payment method
3,A woman called in a problem with her camera an...,product details inquiry
4,A customer has contacted best buy to ask how m...,product details inquiry


In [ ]:
print(len(input_data_sample))

379328


In [8]:
labels_list = ['troubleshooting', 'schedule installation', 'trade in inquiry', 'account security', 'product availability and stock', 'schedule repair', 'product details inquiry', 'check warranty coverage', 'renewal of a plan subscription or membership', 'cancel order', 'unauthorized charge or payment', 'defective product', 'refund request', 'employment or career inquiries', 'rewards or discounts', 'damaged product', 'reschedule delivery', 'price match', 'lost or forgot items', 'transfer call to the right department or store', 'miscellaneous inquiries', 'reschedule repair', 'billing or charge disputes', 'change payment method', 'delivery delays', 'software error', 'software installation', 'warranty claim', 'change or update order', 'delivery tracking', 'refund status', 'schedule delivery', 'change shipping address', 'login issues', 'schedule order pickup', 'return request', 'product compatibility', 'cancellation of a plan subscription or membership', 'change shipping time', 'bad customer service', 'software update', 'reschedule installation', 'payment method', 'network or connectivity issues', 'device damaged', 'website or app complaints', 'payment failed', 'delivery or parts of delivery items missing', 'screen issues', 'forgot my password', 'performance issues', 'fraud concerns', 'best buy credit card', 'reschedule order pickup', 'customer feedback', 'incomplete installation', 'account cancellation']

In [ ]:
# !pip install transformers torch

In [ ]:
# import torch
# print(torch.cuda.is_available())

In [ ]:
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# filtered_data = input_data_sample.dropna()
# filtered_data = filtered_data.astype(str)
# texts = []
# labels = []

# for index, row in filtered_data.iterrows():
#     text = row['text']
#     label = row['label']

#     if len(text) > 20:
#         texts.append(text)
#         labels.append(label)

# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(texts)

# train_sequences = tokenizer.texts_to_sequences(texts)

# max_length = max(len(seq) for seq in train_sequences)
# padded_sequences = pad_sequences(train_sequences, maxlen=max_length, padding='post')


In [9]:
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences

filtered_data = input_data_sample.dropna()
filtered_data = filtered_data.astype(str)
texts = []
labels = []

valid_filtered_data = valid_data_sample.dropna()
valid_filtered_data = valid_filtered_data.astype(str)
valid_texts = []
valid_labels = []

test_filtered_data = test_data_sample.dropna()
test_filtered_data = test_filtered_data.astype(str)
test_texts = []
test_labels = []

for index, row in filtered_data.iterrows():
    text = row['text']
    label = row['label']

    if len(text) > 20:
        texts.append(text)
        labels.append(label)

for index, row in valid_filtered_data.iterrows():
    text = row['text']
    label = row['label']

    if len(text) > 20:
        valid_texts.append(text)
        valid_labels.append(label)

for index, row in test_filtered_data.iterrows():
    text = row['Text']
    label = row['Label']

    if len(text) > 20:
        test_texts.append(text)
        test_labels.append(label)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_tokenized_texts = [tokenizer.encode(text, add_special_tokens=True) for text in texts]
valid_tokenized_texts = [tokenizer.encode(text, add_special_tokens=True) for text in valid_texts]
test_tokenized_texts = [tokenizer.encode(text, add_special_tokens=True) for text in test_texts]

train_max_length = max(len(seq) for seq in train_tokenized_texts)
valid_max_length = max(len(seq) for seq in valid_tokenized_texts)
test_max_length = max(len(seq) for seq in test_tokenized_texts)

train_padded_sequences = pad_sequences(train_tokenized_texts, maxlen=train_max_length, padding='post', truncating='post', dtype='long')
valid_padded_sequences = pad_sequences(valid_tokenized_texts,  maxlen=valid_max_length, padding='post', truncating='post', dtype='long')
test_padded_sequences = pad_sequences(test_tokenized_texts,  maxlen=test_max_length, padding='post', truncating='post', dtype='long')

print("Tokenized Texts:", train_tokenized_texts)
print("Padded Sequences:", train_padded_sequences)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# print(len(filtered_data))
# print(max_length)
# print(len(padded_sequences))
# print(padded_sequences[222952])

375328
67
375271
[   1    6    7   34   34   35    2   36    1   95   10    1   50   48
   39  224 1162   64   52   17   26   28   83  288  530    2  281  722
  520    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0]


In [10]:
label_dict = {}
for idx,i in enumerate(labels_list):
    label_dict[i] = idx
label_codes = list(map(label_dict.get, labels))

In [11]:
valid_label_dict = {}
for idx,i in enumerate(labels_list):
    valid_label_dict[i] = idx
valid_label_codes = list(map(valid_label_dict.get, valid_labels))

In [12]:
test_label_dict = {}
for idx,i in enumerate(labels_list):
    test_label_dict[i] = idx
test_label_codes = list(map(test_label_dict.get, test_labels))

In [ ]:
np.array(label_codes)

array([ 6,  5, 11, ...,  5, 11, 11])

In [ ]:
# print(padded_sequences.shape)
# print(len(label_codes))
# print(len(labels))

(375271, 67)
375271
375271


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_samples = np.bincount(label_codes)

total_samples = len(label_codes)

class_weights = compute_class_weight('balanced', classes=np.unique(label_codes), y=label_codes)
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
print(class_weight_dict)

{0: 0.4679580463704574, 1: 0.4551470276105057, 2: 1.9980885445784415, 3: 3.916538818789985, 4: 0.1753876539609453, 5: 0.1880090740300978, 6: 0.15555113418513797, 7: 3.1055196954650777, 8: 0.5358702388398148, 9: 0.8461254021830054, 10: 1.1952980672450917, 11: 0.34623727343602234, 12: 2.1800336935052864, 13: 2.5717584978070174, 14: 1.0291858299806105, 15: 0.7347061437770299, 16: 0.9817628622704988, 17: 0.9826420528934277, 18: 3.3117212044194995, 19: 1.1296674252549699, 20: 0.909476689375047, 21: 1.7745826831229017, 22: 2.2167345974363517, 23: 1.7736265502117363, 24: 3.2288875695860546, 25: 0.8850251047702601, 26: 1.0791184649050918, 27: 2.2063343680918113, 28: 0.27405826725995774, 29: 1.2151535168671033, 30: 4.38037375541315, 31: 1.8323689825733274, 32: 2.2137531117639426, 33: 1.9606020709904601, 34: 0.7719195397333761, 35: 0.625529857898904, 36: 0.6097149244661942, 37: 1.0423846982878349, 38: 8.539172184676998, 39: 2.424043355812211, 40: 7.15619755911518, 41: 4.41859178146709, 42: 2.512

In [ ]:
# Many - to - one LSTM
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.layers import Bidirectional
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

vocab_size = 10000
# 128 0.4
# 256 0.42
embedding_dim = 200
# Many-to-One LSTM
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=train_padded_sequences.shape[1]))
model.add(Bidirectional(LSTM(units=128, activation='tanh', kernel_initializer='glorot_uniform', return_sequences=True)))
model.add(Dropout(0.3))
model.add(LSTM(units=128, activation='tanh', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=64, activation='tanh', return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(units=57,activation='softmax'))
# 0.001 14.6
# 0.01 0.46
# 0.1 2.3
#0.0001 8

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

model.fit(train_padded_sequences, np.array(label_codes),
          epochs=30,
          batch_size=128,
          callbacks=[early_stopping])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 74, 200)           2000000   
                                                                 
 bidirectional (Bidirection  (None, 74, 256)           336896    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 74, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 74, 128)           197120    
                                                                 
 dropout_1 (Dropout)         (None, 74, 128)           0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                        

In [13]:
from keras.models import load_model

model = load_model('lstm_model_1_17_16_15.h5')

all_predictions = []

predicted = model.predict(train_padded_sequences)
predicted_labels = np.argmax(predicted, axis=1)
all_predictions.append(predicted_labels)

11728/11728 [==============================] - 78s 7ms/step


In [14]:
all_predictions[0][900:1100]

array([20, 25,  6,  5,  1, 24,  4, 17, 37, 20, 11, 28,  6,  6,  0,  0, 49,
       10,  6, 20,  5, 28, 11,  6,  0,  6,  4,  6, 28, 25, 11,  0, 28, 34,
       24,  4,  4, 28, 10,  0, 28,  4, 14, 19, 28,  6, 28,  6, 23, 25,  5,
        1,  6,  6, 11,  5,  6, 19,  6, 28,  5,  5,  1, 37,  6, 35,  5,  9,
       17,  2, 11,  8, 28,  8,  5, 25,  5, 11, 18, 11,  6,  6, 36, 28, 33,
        6,  0,  6,  6,  6,  5,  4,  4, 20, 18, 49, 28,  6,  5, 18,  6,  4,
        6, 29, 28, 34,  3,  0,  9, 11,  4, 14,  1,  6, 14,  8,  4, 15,  4,
       28, 28, 17,  6, 28, 35, 11,  5, 11,  1, 29,  8,  4, 41,  5,  4, 51,
        4,  5, 37, 35,  6,  4,  6,  5,  6, 20,  1,  5, 36,  6,  0,  5,  4,
        4, 30,  8, 34,  6,  2,  4,  6,  6,  6,  1,  4, 28,  5, 28, 19, 15,
        4, 28,  5,  6, 11,  6,  5, 17,  0, 36, 14, 11, 26, 19, 20,  4, 16,
       51, 28, 11,  0, 12, 19, 11, 16,  7, 11, 28, 11,  6])

In [15]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
f1 = f1_score(label_codes, all_predictions[0].tolist(), average='weighted')
print(f'F1 Score: {f1}')
acc1 = accuracy_score(label_codes, all_predictions[0].tolist())
print(f'Accuracy Score: {acc1}')

F1 Score: 0.43235066169576103
Accuracy Score: 0.4449637728468227


In [ ]:
model.save("lstm_model_1_17_16_15.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# error = np.sqrt(mean_squared_error(label_codes, all_predictions[0].tolist()))
# print(f"Root Mean Square Error: {error}")
# r2_value = r2_score(label_codes, all_predictions[0].tolist())
# print(f"R2 Score: {r2_value}")
# accuracy = model.evaluate(label_codes, all_predictions[0].tolist(), verbose=2)[1]
# print(f'Accuracy: {accuracy}')


In [16]:
valid_all_predictions = []
predicted = model.predict(valid_padded_sequences)
predicted_labels = np.argmax(predicted, axis=1)
valid_all_predictions.append(predicted_labels)


1482/1482 [==============================] - 9s 5ms/step


In [17]:

f1 = f1_score(valid_label_codes, valid_all_predictions[0].tolist(), average='weighted')
print(f'F1 Score: {f1}')
acc2 = accuracy_score(valid_label_codes, valid_all_predictions[0].tolist())
print(f'Accuracy Score: {acc2}')

F1 Score: 0.34410768307870576
Accuracy Score: 0.3642787832763785


In [18]:
test_all_predictions = []
predicted = model.predict(test_padded_sequences)
predicted_labels = np.argmax(predicted, axis=1)
test_all_predictions.append(predicted_labels)

1482/1482 [==============================] - 9s 6ms/step


In [19]:
print(len(test_label_codes))
print(len(test_all_predictions[0]))
f1 = f1_score(test_label_codes, test_all_predictions[0].tolist(), average='weighted')
print(f'F1 Score: {f1}')
acc3 = accuracy_score(test_label_codes, test_all_predictions[0].tolist())
print(f'Accuracy Score: {acc3}')


47412
47412
F1 Score: 0.34838138956538833
Accuracy Score: 0.3678182738547203
